In [1]:
# !python -m pip install gensim

In [2]:
from gensim.models import Word2Vec, KeyedVectors

In [3]:
import pandas as pd
import nltk
import json
import string
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Antonio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
pd.set_option('mode.chained_assignment',None)

## Trazendo o Keyed Vectors do arquivo texto

In [6]:
modelo = KeyedVectors.load_word2vec_format("C:/Users/Antonio/Downloads/en_wiki_word2vec_300/en_wiki_word2vec_300.txt")

## Carregando o Dataset

In [7]:
data_set = pd.read_csv("../../dados/movies_metadata.csv")

C:\Users\Antonio\AppData\Local\Temp\ipykernel_14020\1171913871.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_set = pd.read_csv("../../dados/movies_metadata.csv")


In [8]:
data_set = data_set[["genres", "overview", "original_title", "original_language"]]

In [9]:
movies = data_set.query("original_language == 'en'")

In [53]:
movies["overview"].dropna(inplace=True)
movies["genero"].dropna(inplace=True)

## Acessando o apenas o primeiro genero

In [54]:
def extrair_primeiro_genero( texto ):
    # print(texto)
    txt = texto.replace("'", '"')
    lista = json.loads(txt)
    if len(lista):
        return lista[0]['name']
    else:
        return "unknown"

In [55]:
movies["genero"] = movies["genres"].apply(extrair_primeiro_genero)

In [56]:
lista_generos = list(movies["genero"].unique())

In [57]:
def extrair_genero_id( genero ):
    global lista_generos
    num_genero = 11
    try:
        num_genero = lista_generos.index(genero)
    except: 
        pass
    return num_genero

In [58]:
movies["genero_id"] = movies["genero"].apply(extrair_genero_id)

## Limpar o texto de descrição do filme (overview)

### Remover os caracteres com \n \r

In [59]:
movies["overview"].dropna(inplace=True)

In [60]:
trans = str.maketrans('\n\r\t', '   ')
movies["overview_no_enter"] = movies["overview"].str.translate(trans)

### Remover contrações textuais do inglês

In [61]:
contraction_dict = {
                    "aren't": "are not",
                    "can't": "can not",
                    "could've": "could have",
                    "couldn't": "could not",
                    "daren't": "dare not",
                    "didn't": "did not",
                    "doesn't": "does not",
                    "don't": "do not",
                    "hadn't": "had not",
                    "hasn't": "has not",
                    "haven't": "have not",
                    "he's": "he is",
                    "how'd": "how had",
                    "how're": "how are",
                    "how's": "how is",
                    "how've": "how have",
                    "i'd": "i had",
                    "i'm": "i am",
                    "i've": "i have",
                    "isn't": "is+ not",
                    "it's": "it is",
                    "might've": "might have",
                    "mightn't": "might not",
                    "must've": "must have",
                    "mustn't": "must not",
                    "needn't": "need not",
                    "oughtn't": "ought not",
                    "shan't": "shall not",
                    "she'd": "she had",
                    "she's": "she is",
                    "should've": "should have",
                    "shouldn't": "should not",
                    "that'd": "that had",
                    "thats's": "that is",
                    "there'd": "there had",
                    "there's": "there is",
                    "they'd": "they had",
                    "they're": "you are",
                    "they've": "they have",
                    "wasn't": "was+ not",
                    "we'd": "we had",
                    "we're": "we are",
                    "we've": "we have",
                    "weren't": "were not",
                    "what'd": "what had",
                    "what're": "what are",
                    "what's": "what is",
                    "what've": "what have",
                    "when'd": "when had",
                    "when're": "when are",
                    "when's": "when is",
                    "when've": "when have",
                    "where'd": "where had",
                    "where're": "where are",
                    "where's": "where is",
                    "where've": "where have",
                    "who'd": "who had",
                    "who're": "who are",
                    "who's": "who is",
                    "who've": "who have",
                    "why'd": "why had",
                    "why're": "why are",
                    "why's": "why is",
                    "why've": "why have",
                    "would've": "would have",
                    "wouldn't": "would not",
                    "you're": "you are",
                    "you've": "you have",
                    "'cause": "because", 
                    "ain't": "is not", 
                    "aren't": "are not",
                    "can't": "cannot", 
                    "could've": "could have",
                    "he's": "he is",
                    "how'll": "how will",
                    "i'll": "i will",
                    "it'll": "it will",
                    "it's": "it is", 
                    "she'll": "she will",
                    "she's": "she is",
                    "that'll": "that will",
                    "there'll": "there will",
                    "they'll": "they will",
                    "they're": "they are",
                    "we'll": "we will",
                    "we're": "we are",
                    "what'll": "what will",
                    "when'll": "when will",
                    "where'll": "where will",
                    "who'll": "who will",
                    "yo're": "you are",
                    "you'll": "you will"
}

In [62]:
def remove_contractions( text ):
    # print(text)
    # print(type(text))
    if (type(text) == str):
        text = text.lower()
        # print(text)
        for chave in list(contraction_dict.keys()):
            valor = contraction_dict[chave]
            text = text.replace(chave, valor)
        text = text.replace("\'s", " have")        
        return text
    else:
        return ""

In [63]:
movies["overview_no_contraction"] = movies["overview_no_enter"].apply(remove_contractions)

### Remover as pontuções

In [64]:
translator = str.maketrans('', '', string.punctuation)
movies["overview_no_punctuation"]  = movies["overview_no_contraction"] .str.translate(translator)

In [65]:
stopwords = nltk.corpus.stopwords.words('english')

In [66]:
def remove_stop_words( text ):
    nova_lista = []
    palavras = text.split(" ")
    for palavra in palavras:
        if palavra not in stopwords:
            nova_lista.append(palavra)
    return nova_lista

In [67]:
movies["overview_no_stops"] = movies["overview_no_punctuation"].apply(remove_stop_words)

In [68]:
textos = movies["overview_no_stops"]

## Preparar matriz de Word2Vec

In [29]:
def vetorizador_texto( lista_palavras ): 
    vetor = np.zeros(300)
    w2v_unkown = modelo["unknown"]
    for palavra in lista_palavras: 
        try:
            w2v = modelo[palavra]
            vetor += w2v
        except:
            vetor += w2v_unkown
    return vetor

In [31]:
height = len(textos)
width = 300
matriz_w2v_textos = np.zeros((height, width))

for i in range(height):
    try:
        lista_palavras = textos[i]
        w2v = vetorizador_texto(lista_palavras)
        matriz_w2v_textos[i] = w2v
    except:
        pass
    
print(matriz_w2v_textos.shape)

(32269, 300)


In [85]:
dict_w2v_generos.keys()

dict_keys(['animation', 'adventure', 'romance', 'comedy', 'action', 'family', 'history', 'drama', 'crime', 'science', 'fantasy', 'unknown', 'music', 'horror', 'documentary', 'mystery', 'thriller', 'western', 'tv', 'war', 'foreign'])

In [82]:
def choose_genero(w2v, lista_generos):
    errors = 0 
    vetor_vazio = np.zeros(300)
    comparison = w2v == vetor_vazio
    if not comparison.all():
        less_distance = 1000000
        choose_genero = "unknown"
        # print(less_distance)
        for genero in dict_w2v_generos.keys():
            if genero in lista_generos:
                gen_model = dict_w2v_generos[genero]
                cosine_similarity = np.dot(w2v, gen_model)/(np.linalg.norm(w2v)* np.linalg.norm(gen_model))
                # print(i, "Genero: ", genero, "   Distance: ", cosine_similarity)
                if cosine_similarity < less_distance:
                    less_distance = cosine_similarity
                    choose_genero = genero
        print(i, "Choose Genero: ", choose_genero, "    Distance: ", less_distance)
        return choose_genero
    return None
            

In [83]:
movies["genres"][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [89]:
height = len(textos)
# height = 10
for i in range(height):
    try: 
        lista_palavras = textos[i]
        w2v = vetorizador_texto(lista_palavras)
        txt_generos = movies["genres"][i]
        txt = txt_generos.replace("'", '"')
        lista = json.loads(txt)
        lista_generos = []
        for gen in lista:
            lista_generos.append(gen['name'].lower())
        genero_escolhido = choose_genero(w2v, lista_generos)
        if genero_escolhido is not None:
            matriz_w2v_textos[i] = w2v
            movies["genero"][i] = genero_escolhido
    except:
        pass


0 Choose Genero:  family     Distance:  0.38760465175694453
1 Choose Genero:  family     Distance:  0.4284786156357168
2 Choose Genero:  comedy     Distance:  0.4060711813955403
3 Choose Genero:  drama     Distance:  0.3670643884758953
4 Choose Genero:  comedy     Distance:  0.30855101881632696
5 Choose Genero:  thriller     Distance:  0.3213471943135032
6 Choose Genero:  comedy     Distance:  0.3941676922473062
7 Choose Genero:  drama     Distance:  0.3946303995255812
8 Choose Genero:  thriller     Distance:  0.4364384920005049
9 Choose Genero:  thriller     Distance:  0.3185777397070855
10 Choose Genero:  comedy     Distance:  0.3331086253110074
11 Choose Genero:  comedy     Distance:  0.37600538940095113
12 Choose Genero:  animation     Distance:  0.24133591308434318
13 Choose Genero:  drama     Distance:  0.3967126050684334
14 Choose Genero:  adventure     Distance:  0.473569044565227
15 Choose Genero:  drama     Distance:  0.3469371175801503
16 Choose Genero:  drama     Distance: 

391 Choose Genero:  thriller     Distance:  0.27604485972232107
392 Choose Genero:  thriller     Distance:  0.4058138220079965
393 Choose Genero:  documentary     Distance:  0.5166756359640822
394 Choose Genero:  drama     Distance:  0.3661836195680924
395 Choose Genero:  thriller     Distance:  0.4442484224901041
396 Choose Genero:  thriller     Distance:  0.3290004364636051
397 Choose Genero:  comedy     Distance:  0.5212256916657653
398 Choose Genero:  unknown     Distance:  1000000
399 Choose Genero:  documentary     Distance:  0.35001610631116614
400 Choose Genero:  fantasy     Distance:  0.43212275906750064
401 Choose Genero:  unknown     Distance:  1000000
402 Choose Genero:  thriller     Distance:  0.4272319936203715
403 Choose Genero:  drama     Distance:  0.3327600021536376
404 Choose Genero:  drama     Distance:  0.36569914248868035
405 Choose Genero:  fantasy     Distance:  0.3635813970880671
406 Choose Genero:  comedy     Distance:  0.5309730507407056
407 Choose Genero:  d

780 Choose Genero:  drama     Distance:  0.39849047449947816
781 Choose Genero:  drama     Distance:  0.30054418094900887
782 Choose Genero:  drama     Distance:  0.4480562599103804
783 Choose Genero:  comedy     Distance:  0.3874014749334314
784 Choose Genero:  thriller     Distance:  0.2655088193500627
785 Choose Genero:  drama     Distance:  0.22100664344438195
787 Choose Genero:  adventure     Distance:  0.4766888459827992
788 Choose Genero:  thriller     Distance:  0.2949651818209597
789 Choose Genero:  family     Distance:  0.2113792634003924
791 Choose Genero:  comedy     Distance:  0.3866715190156887
792 Choose Genero:  unknown     Distance:  1000000
793 Choose Genero:  thriller     Distance:  0.3207080991772313
794 Choose Genero:  war     Distance:  0.37923625974972963
795 Choose Genero:  comedy     Distance:  0.37030329741156287
796 Choose Genero:  comedy     Distance:  0.20926037992894828
803 Choose Genero:  thriller     Distance:  0.31746873753324667
804 Choose Genero:  fam

1119 Choose Genero:  unknown     Distance:  1000000
1120 Choose Genero:  unknown     Distance:  1000000
1121 Choose Genero:  comedy     Distance:  0.2988721755032288
1123 Choose Genero:  drama     Distance:  0.3518890933512481
1126 Choose Genero:  drama     Distance:  0.3613501286502485
1127 Choose Genero:  comedy     Distance:  0.33765782185968807
1128 Choose Genero:  horror     Distance:  0.4436864467512081
1129 Choose Genero:  documentary     Distance:  0.3473283710892056
1130 Choose Genero:  action     Distance:  0.5493941385363957
1131 Choose Genero:  drama     Distance:  0.4806009640151342
1133 Choose Genero:  drama     Distance:  0.42950491843982463
1137 Choose Genero:  drama     Distance:  0.3549458102319819
1138 Choose Genero:  drama     Distance:  0.29558331807539695
1139 Choose Genero:  drama     Distance:  0.3317838850226606
1140 Choose Genero:  foreign     Distance:  0.3488301972571439
1141 Choose Genero:  western     Distance:  0.508381066130768
1142 Choose Genero:  drama

1482 Choose Genero:  thriller     Distance:  0.3433483287457812
1483 Choose Genero:  unknown     Distance:  1000000
1484 Choose Genero:  foreign     Distance:  0.2858565694355176
1485 Choose Genero:  drama     Distance:  0.3676724831476865
1486 Choose Genero:  thriller     Distance:  0.4043532816135089
1487 Choose Genero:  drama     Distance:  0.348976516770561
1488 Choose Genero:  comedy     Distance:  0.3387372285548216
1489 Choose Genero:  unknown     Distance:  1000000
1490 Choose Genero:  comedy     Distance:  0.5360695939127498
1491 Choose Genero:  fantasy     Distance:  0.3929390602025318
1492 Choose Genero:  comedy     Distance:  0.34587693822120197
1493 Choose Genero:  drama     Distance:  0.3155934911129476
1494 Choose Genero:  thriller     Distance:  0.3302290163307174
1495 Choose Genero:  comedy     Distance:  0.3755265606877904
1496 Choose Genero:  drama     Distance:  0.3612446482556463
1497 Choose Genero:  drama     Distance:  0.4466689974571348
1498 Choose Genero:  come

...

29069 Choose Genero:  drama     Distance:  0.31705171769064244
29070 Choose Genero:  thriller     Distance:  0.2294151936769334
29071 Choose Genero:  drama     Distance:  0.30666563762221327
29072 Choose Genero:  drama     Distance:  0.3412103243675793
29073 Choose Genero:  drama     Distance:  0.3441515048567979
29074 Choose Genero:  thriller     Distance:  0.42929754008350374
29077 Choose Genero:  thriller     Distance:  0.2992667156983834
29078 Choose Genero:  history     Distance:  0.39649264546689067
29081 Choose Genero:  comedy     Distance:  0.3470512263894572
29082 Choose Genero:  horror     Distance:  0.414784362996699
29083 Choose Genero:  thriller     Distance:  0.29967134287311475
29086 Choose Genero:  thriller     Distance:  0.30772798018826064
29088 Choose Genero:  thriller     Distance:  0.3622737377599598
29090 Choose Genero:  thriller     Distance:  0.2536781838302004
29091 Choose Genero:  family     Distance:  0.4383588524661449
29092 Choose Genero:  horror     Distan

29785 Choose Genero:  thriller     Distance:  0.28432174125239174
29788 Choose Genero:  drama     Distance:  0.38558874254584913
29789 Choose Genero:  comedy     Distance:  0.262405143614379
29791 Choose Genero:  thriller     Distance:  0.3978542561671883
29793 Choose Genero:  drama     Distance:  0.33096489939344886
29794 Choose Genero:  documentary     Distance:  0.4774023340597927
29795 Choose Genero:  unknown     Distance:  1000000
29796 Choose Genero:  comedy     Distance:  0.4317898474577635
29798 Choose Genero:  comedy     Distance:  0.3374050277686866
29799 Choose Genero:  comedy     Distance:  0.30794959506446656
29800 Choose Genero:  documentary     Distance:  0.5103788538956828
29802 Choose Genero:  western     Distance:  0.4597946400705639
29804 Choose Genero:  documentary     Distance:  0.36943763996517864
29805 Choose Genero:  documentary     Distance:  0.43371299168660005
29806 Choose Genero:  unknown     Distance:  1000000
29811 Choose Genero:  thriller     Distance:  0

30595 Choose Genero:  thriller     Distance:  0.34698318088232294
30596 Choose Genero:  fantasy     Distance:  0.40304704576816724
30598 Choose Genero:  documentary     Distance:  0.41560518241112004
30600 Choose Genero:  unknown     Distance:  1000000
30601 Choose Genero:  unknown     Distance:  1000000
30602 Choose Genero:  music     Distance:  0.5558659809797103
30603 Choose Genero:  unknown     Distance:  1000000
30604 Choose Genero:  documentary     Distance:  0.46207889047925044
30609 Choose Genero:  unknown     Distance:  1000000
30610 Choose Genero:  comedy     Distance:  0.5658523637350001
30611 Choose Genero:  comedy     Distance:  0.5299202673523293
30612 Choose Genero:  comedy     Distance:  0.3870250085483255
30613 Choose Genero:  family     Distance:  0.47999019618377614
30614 Choose Genero:  horror     Distance:  0.5043002246787253
30615 Choose Genero:  documentary     Distance:  0.36969455921887345
30616 Choose Genero:  drama     Distance:  0.35466377376777664
30617 Cho

31314 Choose Genero:  unknown     Distance:  1000000
31315 Choose Genero:  unknown     Distance:  1000000
31316 Choose Genero:  documentary     Distance:  0.39258415157404025
31317 Choose Genero:  unknown     Distance:  1000000
31318 Choose Genero:  unknown     Distance:  1000000
31319 Choose Genero:  unknown     Distance:  1000000
31320 Choose Genero:  thriller     Distance:  0.5086026680679767
31322 Choose Genero:  drama     Distance:  0.2862950433761412
31323 Choose Genero:  drama     Distance:  0.32774552631525244
31324 Choose Genero:  thriller     Distance:  0.36932883693450375
31326 Choose Genero:  drama     Distance:  0.3536233045140104
31327 Choose Genero:  thriller     Distance:  0.2582389746094215
31330 Choose Genero:  fantasy     Distance:  0.41616199137170146
31337 Choose Genero:  fantasy     Distance:  0.28415400750533154
31344 Choose Genero:  unknown     Distance:  1000000
31345 Choose Genero:  documentary     Distance:  0.4123222427094908
31346 Choose Genero:  comedy    

32122 Choose Genero:  drama     Distance:  0.3327131369586586
32123 Choose Genero:  drama     Distance:  0.3203889782640159
32124 Choose Genero:  drama     Distance:  0.2963557788521639
32125 Choose Genero:  comedy     Distance:  0.37156060973749305
32126 Choose Genero:  comedy     Distance:  0.2834124519822374
32128 Choose Genero:  drama     Distance:  0.533465791403816
32130 Choose Genero:  drama     Distance:  0.31181139124163354
32131 Choose Genero:  thriller     Distance:  0.35281638973896484
32133 Choose Genero:  comedy     Distance:  0.5098522673343188
32134 Choose Genero:  documentary     Distance:  0.5007695214097737
32135 Choose Genero:  documentary     Distance:  0.36948359731479724
32136 Choose Genero:  documentary     Distance:  0.48783201766263445
32137 Choose Genero:  documentary     Distance:  0.4495560630507752
32138 Choose Genero:  family     Distance:  0.35440688778323554
32139 Choose Genero:  drama     Distance:  0.3006318728783964
32140 Choose Genero:  unknown     

In [90]:
train_X, test_X, train_Y, test_Y = train_test_split(matriz_w2v_textos, movies["genero"], test_size=0.25, random_state=50)

In [91]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(train_Y)
Test_Y = Encoder.fit_transform(test_Y)

In [92]:
print(train_X.shape)
print(test_X.shape)

(24201, 300)
(8068, 300)


In [93]:
lista_generos

['action', 'science fiction']

In [38]:
# idx = lista_generos.index('Science Fiction')
# lista_generos[idx]='science'
# idx = lista_generos.index('TV Movie')
# lista_generos[idx]='tv'


# dict_w2v_generos = {}
# for genero in lista_generos: 
#     genero = genero.lower()
#     dict_w2v_generos[genero] = modelo[genero]

In [51]:
for genero in dict_w2v_generos.keys():
    print(genero, np.linalg.norm(dict_w2v_generos[genero]))

animation 3.2788527
adventure 3.1866658
romance 2.9361107
comedy 3.4596713
action 3.0095065
family 2.9532094
history 2.6248894
drama 3.4183264
crime 3.108495
science 3.2264402
fantasy 3.4163253
unknown 3.4409328
music 2.8462336
horror 3.3719115
documentary 2.9217546
mystery 2.7685804
thriller 3.7359772
western 3.1287322
tv 2.9879045
war 3.3584478
foreign 3.4518964


In [78]:
# errors = 0 
# vetor_vazio = np.zeros(300)
# # for i in range(len(matriz_w2v_textos)):
# for i in range(55):
#     w2v = matriz_w2v_textos[i]
#     comparison = w2v == vetor_vazio
#     if not comparison.all():
#         less_distance = 1000000
#         choose_genero = "unknown"
#         # print(less_distance)
#         for genero in dict_w2v_generos.keys():
#             gen_model = dict_w2v_generos[genero]
#             cosine_similarity = np.dot(w2v, gen_model)/(np.linalg.norm(w2v)* np.linalg.norm(gen_model))
#             # print(i, "Genero: ", genero, "   Distance: ", cosine_similarity)
#             if cosine_similarity < less_distance:
#                 less_distance = cosine_similarity
#                 choose_genero = genero
#         print(i, "Choose Genero: ", choose_genero, "    Distance: ", less_distance)
    

0 Choose Genero:  science     Distance:  0.22697585832968678
1 Choose Genero:  science     Distance:  0.3255544678249788
2 Choose Genero:  science     Distance:  0.2870006496934407
3 Choose Genero:  science     Distance:  0.25996198228403117
4 Choose Genero:  thriller     Distance:  0.2533441832400287
5 Choose Genero:  thriller     Distance:  0.3213471943135032
6 Choose Genero:  science     Distance:  0.22492624892130925
7 Choose Genero:  science     Distance:  0.27721111130088144
8 Choose Genero:  science     Distance:  0.3114616514230757
9 Choose Genero:  animation     Distance:  0.2399089009527797
10 Choose Genero:  animation     Distance:  0.27975890300554496
11 Choose Genero:  science     Distance:  0.28642889971614655
12 Choose Genero:  music     Distance:  0.2093536793708521
13 Choose Genero:  animation     Distance:  0.3078015602525113
14 Choose Genero:  thriller     Distance:  0.2869797077707812
15 Choose Genero:  foreign     Distance:  0.24399393472689285
16 Choose Genero:  t

In [94]:
# fit the training dataset on the NB classifier
model = SVC()
model.fit(train_X,Train_Y)
# predict the labels on validation dataset
predictions_model = model.predict(test_X)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_model, Test_Y)*100)

Naive Bayes Accuracy Score ->  22.99206742687159


In [115]:
modelo.most_similar(positive=["woman", "king"], negative=["man"])


[('queen', 0.6688153743743896),
 ('princess', 0.552842915058136),
 ('prince', 0.5264531970024109),
 ('nanasipau', 0.4919755756855011),
 ('maathorneferure', 0.49078336358070374),
 ('regnant', 0.49029964208602905),
 ('chandrmondol', 0.4897799491882324),
 ('regent', 0.48750442266464233),
 ('crown', 0.4872990548610687),
 ('elizabeth', 0.48600274324417114)]